# FreeTrace tutorial

# Requirements
FreeTrace requires Linux platform to infer fully the molecular trajectories.</br>
- CUDA accelerates the localisation of molecules from video if you have a NVIDIA GPU in your machine.</br>
- Cuda installation: [Nvidia Cuda installation](https://developer.nvidia.com/cuda-toolkit)</br>
  
You can run FreeTrace without a GPU; however, it'll be very slow if you try to infer the trajectories with fBm mode.</br>
In the case of high molecular density in a heterogeneous medium, we recommend running FreeTrace on a GPU-equipped machine for the best inference result.</br>

# Compatibility

![convert notebook to web app](tmps/compatibility_table.png)

# FreeTrace installation
After downloading FreeTrace, the import of FreeTrace.installation (shown in below cell) will download additional DNN models, packages and compile to run FreeTrace.</br>
This step probably overrides your current versions of packages. Please check [requirements.txt](FreeTrace/requirements.txt).</br>
If you don't want to upgrade your packages to the latest version, please install them manually.</br>
FreeTrace utilises Tensorflow 2.14/2.17 to run it with source code and infer the trajectories under fractional Brownian motion.</br>
Once the installation is successfully finished, you don't need to re-import the FreeTrace.installation for the installation of packages.</br>
#### Please delete the below cell after installation is finished to prevent re-installation when you run this notebook again!

In [ ]:
!pip3 install FreeTrace
import FreeTrace.installation
import os
import subprocess
os.mkdir("inputs")
subprocess.run(['wget', 'https://github.com/JunwooParkSaribu/FreeTrace/blob/main/inputs/sample0.tiff', '-P' f'inputs'])

# FreeTrace algorithm

FreeTrace consists of two major algorithms.
- Localisation of particles from video.</br>
- Reconnection of localised particles .</br>
The units of FreeTrace output are <b>pixels</b> (coordinates) and <b>frames</b> (time). To analyse the result with your microscopy setting, you need to convert the data into micrometres/nanometers and milliseconds.</br>

The detailed algorithm of FreeTrace will be available soon.

### ---------------------------------------------------------------------------------------------------------------

# 1. Parameter settings
Below is the parameters of FreeTrace.</br>

In [ ]:
from FreeTrace import Tracking, Localization

"""
Path of input video and output results.
"""
video_name = 'inputs/sample0.tiff'  # input video.tiff path.
output_dir = 'outputs'  # output path.
verbose = 1  # print ETA of process if verbose=1, 0 otherwise.


"""
Basic parameters.
"""
window_size = 7  # Length (pixel) of sliding window to localise molecules from video
threshold = 1.0  # Threshold factor to determine the existence of a molecule inside sliding windows, base thresholds are calculated with SNR of images.
cutoff = 3  # Minimum length of trajectory for the output.


"""
Advanced parameters.
"""
gpu_for_localization = True  # GPU acceleration of localisation if True, False otherwise.
realtime_localization = False  # Real-time visualisation of localisation if True, False otherwise.
save_localization_video = False  # Save the visualised localisation in tiff format (requires enough RAM).

fBm_mode = True  # Tracking under fBm if True (slow). Otherwise, FreeTrace infers the trajectories under classical Brownian motion.
jump_threshold = None  # Maximum jump distance of molecules for 1 frame. If None, FreeTrace estimates it automatically with GMM.
graph_depth = 3  # Number of frames for the inference of trajectories at each step, between 1 and 5 inclusive. A higher number will slow down the speed.
realtime_tracking = False  # Real-time visualisation of tracking if True, False otherwise.
save_tracking_video = False  # Save the visualised trajectories in tiff format (requires enough RAM).

# 2. Localization

- Run the localization of molecules from video with pre-defined parameters above.</br>
- Localization.run_process() returns True if it finishes successfully.

In [ ]:
loc = Localization.run_process(input_video_path=video_name,
                               output_path=output_dir,
                               window_size=window_size,
                               threshold=threshold,
                               gpu_on=gpu_for_localization,
                               save_video=save_localization_video, 
                               realtime_visualization=realtime_localization,
                               verbose=verbose)

# 3. Trajectory inference

- Run the tracking of localized molecules from video and localization results, with pre-defined parameters above.</br>
- Tracking.run_process() returns True if it finishes successfully.

In [ ]:
track = Tracking.run_process(input_video_path=video_name,
                             output_path=output_dir,
                             graph_depth=graph_depth,
                             cutoff=cutoff,
                             jump_threshold=jump_threshold,
                             gpu_on=fBm_mode,
                             save_video=save_tracking_video,
                             realtime_visualization=realtime_tracking,
                             verbose=verbose)

Once the localization and tracking of FreeTrace finish successfully, you can find the molecular trajectory file in your output directory with the name <b>video_traces.csv</b>.</br></br>
If your trajectory is heterogeneous, you need an additional inference with a program of your choice to segment the trajectory.</br>
Below are the simple analysis steps of trajectory data. We assumed that your trajectory is homogeneous or already segmented with your preferred software.</br>

# 4. Load trajectory data

In [ ]:
from FreeTrace.module.data_load import read_csv, read_multiple_csv
trajectory_data = read_multiple_csv('outputs')
#trajectory_data = trajectory_data[trajectory_data['H'] < 0.35]  # simple classification w.r.t. H or K value.
trajectory_data

# 5. Simple analysis of trajectory

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from FreeTrace.module.preprocessing import simple_preprocessing, trajectory_visualization,\
inv_cdf_cauchy, cdf_cauchy_2mixture, pdf_cauchy_2mixture, func_to_minimise, cdf_cauchy_1mixture, pdf_cauchy_1mixture, cauchy_location


"""
Option settings for data analysis.
"""
PIXELMICRONS = 0.16
FRAMERATE = 0.01
CUTOFF = [3, 99999]  # minimum length of trajectory to consider for the analysis.
number_of_bins = 50
figure_resolution_in_dpi = 200
figure_font_size = 20


"""
Preprocessing generates 4 data.
@params: 
data folder path, pixel microns, frame rate, cutoff

@output: 
analysis_data1: pd.DataFrame, 
analysis_data2: pd.DataFrame, 
analysis_data3: pd.DataFrame,
analysis_data4: Nested list[state_nb][time_lag], contains data of 1d displacements for time lags
analysis_data5: pd.DataFrame, 
MSD: pd.DataFrame, 
TAMSD: pd.DataFrame

Simple_preprocessing includes the following steps.
1. Exclude the trajectory where the length is shorter or longer than CUTOFF
2. Convert from pixel unit to micrometre unit with PIXELMICRONS and FRAMERATE
3. Generate pandas DataFrames for visualizations.
"""
analysis_data1, analysis_data2, analysis_data3, analysis_data4, analysis_data5,\
_, _, msd, tamsd,  total_states, _ = simple_preprocessing(data=trajectory_data,
                                                          pixelmicrons=PIXELMICRONS,
                                                          framerate=FRAMERATE,
                                                          cutoff=CUTOFF,
                                                          div_time_gap=True)

trajectory_image, angle_image, legend_patch, cmap_for_graph, cmap_for_plot = trajectory_visualization(trajectory_data,
                                                                                                      analysis_data1,
                                                                                                      CUTOFF,
                                                                                                      PIXELMICRONS)

print(f'\nanalysis_data1:\n', analysis_data1)
print(f'\nanalysis_data2:\n', analysis_data2)
print(f'\nanalysis_data3:\n', analysis_data3)
print(f'\nanalysis_data5:\n', analysis_data5)
print(f'\nMSD:\n', msd)
print(f'\nEnsemble-averaged TAMSD:\n', tamsd)

# 6. Plots of analysed trajectory data.</br>
Data is stored as</br>
1. analysis_data1: (DataFrame: contains data of mean_jump_distance, duration, traj_id, color_code)</br>
2. analysis_data2: (DataFrame: contains data of displacments, color_code)</br>
3. msd: (DataFrame: contains msd.) </br>
4. tamsd: (DataFrame: contains ensemble-averaged tamsd.) </br>

Units: </br>
mean jump-distance: set of averages of jump distances in um.</br>
duration: duration of trajectory in seconds.</br>
displacement: displacement(time lag=1) of all trajectories in um.</br>

In [ ]:
# Trajectory image.
fig, axs = plt.subplots(nrows=1, ncols=2, dpi=figure_resolution_in_dpi)
axs[0].imshow(trajectory_image)
axs[1].imshow(angle_image)
axs[0].set_xticks([])
axs[0].set_yticks([])
axs[1].set_xticks([])
axs[1].set_yticks([])
plt.tight_layout()

In [ ]:
#p0: H - K distribution plot of H(x-axis) and K(y-axis)
"""
Remark: The unit of K shown in this image is at the pixel and frame scale; it is not converted here to micrometre and second scale. 
"""
fig, axs = plt.subplots(1, 1, layout='constrained', dpi=figure_resolution_in_dpi, num=f'p0')
colormap = sns.color_palette("mako", as_cmap=True)
axs.add_patch(Rectangle((0, -100), 1.0, 1000, ec='none', fc=colormap(0), zorder=0))
sns.kdeplot(
    data=analysis_data1, x="H", y="K", fill=True, ax=axs, thresh=0, levels=100, cmap=colormap, log_scale=(False, True), bw_adjust=1.0,
)
axs.set_yscale('log')
axs.set_ylabel(f"K (generalised diffusion coefficient)")
axs.set_xlabel(f"H (Hurst exponent)")
fig.suptitle(f"Cluster of estimated H and K of individual trajectories for 1 frame.")
axs.set_xlim([0.0, 1.0])
axs.set_ylim([10**-4, (10**1)*2])
axs.set(xlabel="H", ylabel="K (pixel^2 / elapsed time^2H)")
plt.yticks(fontsize=figure_font_size)
plt.xticks(fontsize=figure_font_size)

In [ ]:
#p1: histogram with kde(kernel density estimation) plot of mean jump distance grouped by state.
plt.figure(f'p1', dpi=figure_resolution_in_dpi)
p1 = sns.histplot(analysis_data1, x=f'mean_jump_d', stat='percent', bins=number_of_bins, kde=True)
p1.set_xlabel(r'mean jump-distance($\mu m$)')
p1.set_title(f'mean jump-distance for each state')
plt.yticks(fontsize=figure_font_size)
plt.xticks(fontsize=figure_font_size)
plt.xticks(rotation=90)
plt.tight_layout()

In [ ]:
#p2: displacement histogram
plt.figure(f'p2', dpi=figure_resolution_in_dpi)
p2 = sns.histplot(data=analysis_data2, x='2d_displacement', stat='percent', bins=number_of_bins, kde=True)
p2.set_title(f'displacement histogram')
p2.set_xlabel(r'displacment($\mu m$)')
plt.yticks(fontsize=figure_font_size)
plt.xticks(fontsize=figure_font_size)
plt.xticks(rotation=90)
plt.tight_layout()

In [ ]:
#p3: trajectory length(sec) histogram
plt.figure(f'p3', dpi=figure_resolution_in_dpi)
p3 = sns.histplot(data=analysis_data1, x='duration', stat='percent', bins=number_of_bins, kde=True)
p3.set_title(f'trajectory length histogram')
p3.set_xlabel(r'duration($s$)')
plt.yticks(fontsize=figure_font_size)
plt.xticks(fontsize=figure_font_size)
plt.xticks(rotation=90)
plt.tight_layout()

In [ ]:
#p4: angles histogram
fig, axs = plt.subplots(1, 2, num=f'p4', figsize=(18, 9))
sns.histplot(data=analysis_data3, x='angle', stat='proportion', hue='state', common_norm=False, bins=number_of_bins, kde=True, ax=axs[0], kde_kws={'bw_adjust': 1})
sns.ecdfplot(data=analysis_data3, x='angle', stat='proportion', hue='state', ax=axs[1])
axs[0].set_title(f'angle histogram')
axs[0].set_xlabel(r'Angle (degree)')
axs[1].set_title(f'angle CDF')
axs[1].set_xlabel(r'Angle (degree)')
plt.tight_layout()

In [ ]:
#p5: MSD
plt.figure(f'p5', dpi=figure_resolution_in_dpi)
p5 = sns.lineplot(data=msd, x=msd['time'], y=msd['mean'])
p5.set_title(f'MSD')
p5.set_xlabel(r'time($s$)')
p5.set_ylabel(r'$\frac{\text{MSD}}{\text{2} \cdot \text{dimension}}$ ($\mu m^2$)')
plt.yticks(fontsize=figure_font_size)
plt.xticks(fontsize=figure_font_size)
plt.xticks(rotation=90)
plt.tight_layout()

In [ ]:
#p6: Ensemble-averaged TAMSD
plt.figure(f'p6', dpi=figure_resolution_in_dpi)
p6 = sns.lineplot(data=tamsd, x=tamsd['time'], y=tamsd['mean'])
p6.set_title(f'Ensemble-averaged TAMSD')
p6.set_xlabel(r'lag time($s$)')
p6.set_ylabel(r'$\frac{\text{TAMSD}}{\text{2} \cdot \text{dimension}}$ ($\mu m^2$)')
plt.yticks(fontsize=figure_font_size)
plt.xticks(fontsize=figure_font_size)
plt.xticks(rotation=90)
plt.tight_layout()

In [ ]:
#p7: 1D displacement for the first time lag
fig, axs = plt.subplots(1, 1, num=f'p7', dpi=figure_resolution_in_dpi)
axs.hist(analysis_data4[0][1], bins=100)
axs.set_title(f'1D displacement histogram')
axs.set_xlabel(r'1D displacement (um)')
axs.set_ylabel(r'Count')
plt.tight_layout()
plt.show()

In [ ]:
#p8: 1D ratio distribution with Cauchy fitting for the selected state.
plt.figure(num=f"p8", dpi=figure_resolution_in_dpi)
STATE_TO_PLOT = 0  # always 0 unless you classified the individual trajectories.
vline_ymax = 0.018
hist_bins = np.arange(-10000, 10000, 0.05)
target = analysis_data5[analysis_data5['state']==STATE_TO_PLOT]['1d_ratio'].to_numpy()
hist, bin_edges = np.histogram(target, bins=hist_bins, density=True)
plt.hist(bin_edges[:-1], bin_edges, weights=hist / np.sum(hist), alpha=1.0, histtype='stepfilled', zorder=0, linewidth=2)
xs = hist_bins[:-1] + (hist_bins[1] - hist_bins[0])/2
cons = ({'type': 'ineq', 'fun': lambda k:  k[2] - 0.2},
        #{'type': 'eq', 'fun': lambda k:  k[0] + k[1] + k[2] - 1},
        )
res1 = minimize(func_to_minimise, x0=[0.5, 0.1], args=(pdf_cauchy_1mixture, xs, hist / np.sum(hist)), 
                method='trust-constr',
                #constraints=cons, 
                bounds=((1e-6, 0.9999), (1e-6, None),),
                tol=1e-10,
                )
params, residual = res1.x, res1.fun
plt.vlines(cauchy_location(params[0]), ymin=0, ymax=vline_ymax, colors='red', alpha=0.8, zorder=5, linewidth=5, label=f"loc: {round(cauchy_location(params[0]), 1)}")
plt.plot(xs, pdf_cauchy_1mixture(xs, *params), c='red',
            label=r"$\hat{H}$: %5.4f" % params[0], alpha=0.9, zorder=3, linewidth=5)
plt.xlim([-5, 5])
plt.ylim([0, vline_ymax])
plt.yticks(fontsize=figure_font_size)
plt.xticks(fontsize=figure_font_size)
plt.legend(title=r"H", fontsize="40", title_fontsize="40", loc='upper right')
plt.gca().ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
plt.tight_layout()